### This is the code used to match the crawled forum data to Behavior Change Annotations
Matching was last successfully tested in December, 2021

In [1]:
import pandas as pd
import numpy as np
import re

In [36]:
#data = pd.read_json(path_or_buf="baldNichtraucher.json", encoding='utf-8')
data = pd.read_json(path_or_buf="new_data_allgemein.json")
data = pd.read_json(path_or_buf="new_data_PsychoTherapie.json")
orig_data = pd.read_json(path_or_buf="psychoTherapie.json")

In [37]:
data.head()

,title,thread_id,post_id,date,username,content
0,Intuitives Essen,101827,1943640,"8. Dezember 2021, 12:03",LisaIntuitiv,\n\nHallo ich bin Lisa und ich möchte berichte...
1,Intuitives Essen,101827,1943652,"8. Dezember 2021, 13:32",Frau_vonWegen,"\n\nIst ja Wahnsinn, dass es überhaupt notwend..."
2,Intuitives Essen,101827,1943653,"8. Dezember 2021, 13:40",*Heike*,\n\nAlso ich habe einmal Geld ausgegeben fürs ...
3,Intuitives Essen,101827,1943656,"8. Dezember 2021, 13:59",Anette85,\n\nIch denke wenn du deinen Körper nicht mit ...
4,Intuitives Essen,101827,1943658,"8. Dezember 2021, 14:06",*Heike*,"\n\n2 % können ihr Gewicht halten, daran änder..."


In [38]:
len(data)

32

In [39]:
delimiter = "REPLACE"   
 
def break_line(match):
   return match.group() + delimiter

dict_of_regex_replacements = {r"z\. *b\.": "zb",  r"z\. *B\.": "zb",r"EW\.": "EW",r"Z\. *B\.": "zb", r"zB\.": "zb",
                       r"z\.B\.": "zb", r"zb\.": "zb",r"z\.B": "zb", r"z\.b": "zb", r"u\. *a\.": "ua", r"etc\.": "etc", r"etc\.\)": "etc",
                       r" eig\.": "eig", r"l\.g\.": "lg", r"o\.k\.": "ok", r"ca\.": "ca", r"Ca\.": "ca", r"d\.m\.": "dm", r"d\.M\.": "dm",
                       r"p\. *s\.": "ps", r"P\. *s\.": "ps", r"P\. *S\.": "ps", r"z\.t\.": "zt", r"wg\.": "wg",r"\.-": "-",
                       r"-\.": "-", r"Tg\.": "Tg", r"o\. *ä\.": "oä", r"d\. *h\.": "dh", r"D\. *H\.": "dh", r"d\. *H\.": "dh",
                       r"dH\.": "dh",r"dh\.": "dh", r"z\. *Zt\.": "zzt", r"Dr\.": "Dr", r"v\. *a\.": "va", r"\d\d\.*\d\d\.": "xx/xx/",
                       r"s\. *u\.": "su", r"u\.s\.w\.": "usw", r"usw\.":"usw",r"soz\.": "soz", r"vllt\.": "vlt", r"\(\!\)": " ",
                       r"ank\.tzt": "ankotzt", r"\!\" z\.B\.": "\" zb",r"Gr\.": "Gr", r"Dez\.": "Dez", r"(\d+\.)(\d+\.)\d*": "xx/xx/xxxx",
                       r"(\d+\.)(\d+) ": "12,3 ",r"(\d+\.)(\d+)": "xx/xx", r"\d+\.\d+\.": "xx/xx", r"bz\.w": "bzw", r"Bzw\.": "bzw", 
                       r"bzw\.": "bzw", r"inkl\.": "inkl", r"psych\.": "psych",r"mind\.": "mind", r"Min\.": "Min", r" min\.": "min", r"Verh\.": "Verh",
                       r"Ern\.": "Ern", r"bezgl\.": "bezgl", r"ltd\.": "ltd", r"bspw\.": "bspw", r"ltr\.": "ltr", r"anschl\.": "anschl",
                       r"s\.oliver": "soliver", r" event\.": " event", r"std\.": "std", r"Std\.": "std", r"max\.": "max", r"L\.G\.": "lg",
                       r"LG\.": "lg", r"tägl\.": "tägl", r" u\.": " u", r" u\.":" u", r" od\.": " od",r"Co\.": "Co", r"co\.": " co",r"bzgl\.": "bzgl", r"evtl\.": "evtl",
                       r"pos\.": "evtl", r"M\.O\.B\.I\.L\.I\.S": "mobilis",r"m\.o\.b\.i\.l\.i\.s\.": "mobilis", "fddb\.info":"fddb_info", r"i\.d\.R\.*": "idR",
                       r"z\. *T\.":"zT", r"Vit\.":"Vit",r"ggf\.":"ggf", r"m\.E\.": "mE", r"k\.o": "ko", r"bezw\.":"bzw",r"Dh\.":"dh", r"Nr\.":"Nr", r"Evtl\.":"evtl",
                       r"O\.k\.":"ok", r"D\.h\.":"dh", r"zBsp\.":"zb", r"U\.a\.":"Ua", r"Bzgl\.":"bzgl", "gr\.":"gr", "d\.h":"dh", "i\.d\.R\.":"idR"}

dict_of_non_regex_replacements = {"etc.": "etc", " u.": " u", "z.b.": "zb", " ca.": " ca", "Co.": "Co", "ggf.":"ggf","ua.":"ua", "U.a.":"ua",
                               "Bzgl.": "bzgl", "Mrs.":"Mrs", "Nr.":"Nr", "z.T.":"zT", "gr.": "gr", "(!)": "()", "d.h": "dh", "D.h.":"dh",
                               "Vit.":"Vit", "m.E.": "mE", "k.o": "ko", "bezw.":"bzw", "O.k.":"ok", "Dh.":"dh", "Evtl.":"evtl", "Bsp.":"Bsp", "Bzgl.":"bzgl", "d.h":"dh"}


data["split"] = ""

for key in dict_of_regex_replacements.keys():
    data["content"].replace(key, dict_of_regex_replacements[key], inplace=True, regex=True)
    
for key in dict_of_non_regex_replacements.keys():
    data["content"].replace(key, dict_of_non_regex_replacements[key], inplace=True)

for index, row in data.iterrows():
    data.loc[index, "split"] = re.sub(r"!+|\?+|\.+|\!\?+|\?\!+|\.+\?+", break_line, data.loc[index,"content"])

data = data.replace(r'\n',' ', regex=True) 
data.drop(columns="content", inplace=True)
data = data.assign(split=data['split'].str.split(delimiter)).explode('split', ignore_index=True)
len(data)

175

In [40]:
data.head()

,title,thread_id,post_id,date,username,split
0,Intuitives Essen,101827,1943640,"8. Dezember 2021, 12:03",LisaIntuitiv,"Hallo ich bin Lisa und ich möchte berichten,..."
1,Intuitives Essen,101827,1943640,"8. Dezember 2021, 12:03",LisaIntuitiv,Ich war mein Leben lang unzufrieden mit mein...
2,Intuitives Essen,101827,1943640,"8. Dezember 2021, 12:03",LisaIntuitiv,Durch Zufall bin ich dann auf die Ernährungsf...
3,Intuitives Essen,101827,1943640,"8. Dezember 2021, 12:03",LisaIntuitiv,Ich recherchierte im Internet und las Bücher ...
4,Intuitives Essen,101827,1943640,"8. Dezember 2021, 12:03",LisaIntuitiv,"Leider gelang es mir nie zu 100%, immer wiede..."


In [41]:
nan_value = float("NaN")

list_to_replace = ["", " ", "  ", "    ", "   ", "           ", "      ", "!", "?", ".", "!!", "??", "..",
                  "!!!", "???", "...", " !", " ?", " .", " !!", " ??", " ..", " !!!", " ???", "????", "  ...", "  ....",
                  " ....", "....", ".......", " ......", "    ", ".......", " ......", "  ....", "      ", " ...", "                   ",
                  " .....", ".....", "    ...", "     ...", "  .", "   .", "  ????", "     ", "        "]
                   
for item in list_to_replace:
    data["split"].replace(item, nan_value, inplace=True) 

data.dropna(subset = ["split"], inplace=True)
data.reset_index(inplace=True)

# Making sure that both datasets have the same length
len(data)

149

In [42]:
data.to_csv("new_data_PsychoTherapie.csv")

In [43]:
dat_psy = pd.read_csv("new_data_PsychoTherapie.csv")
dat_allg = pd.read_csv("new_data_allgemein.csv")

In [44]:
len(dat_psy) + len(dat_allg)

992

### We can now look at the posts, that have a single label and polarization

These were used for the testing Inter-Rater reliability and conducting machine learning experiments

In [2]:
forum_splitted = pd.read_csv("baldNichtraucher_annotiert.csv", sep=";")

In [3]:
categories = ["C", "TS", "R"]
polarization = ["+", "-"]

forum_splitted = forum_splitted[forum_splitted["label"].isin(categories)]
forum_splitted = forum_splitted[forum_splitted["valence"].isin(polarization)]

In [4]:
len(forum_splitted)

683

In [5]:
forum_splitted.to_csv("Nichtraucher_annotations_classifier_v2.csv")

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
forum_splitted = pd.read_csv("Nichtraucher_annotations_classifier_v2.csv")

In [8]:
forum_splitted["valence"].replace("+", 1, inplace=True)
forum_splitted["valence"].replace("-", 0, inplace=True)

In [9]:
forum_splitted["label"].replace("R", 0, inplace=True)
forum_splitted["label"].replace("TS", 1, inplace=True)
forum_splitted["label"].replace("C", 2, inplace=True)

forum_splitted["sublabel"].fillna(0, inplace=True)
forum_splitted["sublabel"].replace("a", 1, inplace=True)
forum_splitted["sublabel"].replace("d", 2, inplace=True)
forum_splitted["sublabel"].replace("n", 3, inplace=True)

In [10]:
sublabels_forum_splitted = forum_splitted[forum_splitted["label"] == 0]

In [11]:
valence = forum_splitted[["sentence", "valence"]].copy()
valence.rename(columns={"valence":"labels"}, inplace=True)

In [12]:
labels = forum_splitted[["sentence", "label"]].copy()
labels.rename(columns={"label":"labels"}, inplace=True)

In [13]:
sublabels = sublabels_forum_splitted[["sentence", "sublabel"]].copy()
sublabels.rename(columns={"sublabel":"labels"}, inplace=True)

In [14]:
valence.to_csv("nichtraucher_valence_v2.csv")
labels.to_csv("nichtraucher_labels_v2.csv")
sublabels.to_csv("nichtraucher_sublabels_v2.csv")